## Table of Contents

1. [Introduction](#0)<br>
2. [Data acquisition and cleaning](#2)<br>
3. [Data Analysis](#4) <br>
4. [Conclusions](#6) <br>
5. [Future directions](#8) <br>


# 1. Introduction <a id="0"></a>

The city of Rio de Janeiro is going through a fiscal crisis, and has not been able to provide good social care policies to the population in need. Unfortunately, the city also has high inequality rates. In a situation like this where the government cannot expend, we have to be clever to propose solutions in order to help the poor without compromising the city's budget. 

In this work we study data from the census of 2010 that clearly show the inequality problem of the city. Then, we propose the creation of system in which the supermarkets and their clients can sell products that are near to the expiration date or have any imperfections, at discount, to food banks and the creation of community refectories in which the poor can have meals paying affordable prices.

# 2. Data acquisition and cleaning <a id="2"></a>

For this work we will use 3 data sets:
* Excel table with the data of human development from "Atlas do Desenvolvimento Humano no Brasil", an organization that use the census data to generate a study. The excel file can be obtained in <a href="http://www.atlasbrasil.org.br/2013/data/rawData/dados_rj.zip">here</a>.
* GeoJson with the neighbourhoods of Rio de Janeiro, that you can find <a href="https://pgeo3.rio.rj.gov.br/arcgis/rest/services/Limites/Limites/MapServer/3/query?where=1%3D1&outFields=*&outSR=4326&f=json">here</a>.
* Location and amount of supermarkets in the city, that we will get with the foursquare API.

At the beginning of the analysis we will focus on structuring the data. The excel file containing the human development data has a lot of sub regions, which made a challenge to work with, as you will see. We get the official list of neighbourhoods from the geojson and match to the human development data, so we can visualize it in Folium choropleths. 

Then we explore the supermarkets to conclude that the system is viable.

# 3. Data Analysis <a id="4"></a>

First, we will import and install all dependencies we need.

In [1]:
import pandas as pd
import numpy as np
import json

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from folium import plugins

! pip install unidecode
from unidecode import unidecode

!conda install -c anaconda xlrd --yes

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler

from IPython.display import IFrame

import requests # library to handle requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - xlrd


The following packages will be UPDATED:

    openssl: 1.0.2r-h14c3975_0 conda-forge --> 1.0.2s-h7b6447c_0 anaconda

The following packages will be DOWNGRADED:

 

Now we load the human development data into pandas dataframe. 

In [2]:
# load the human development excel table in pandas dataframe
df_hd_RJ = pd.read_excel('HDI_RJ.xlsx',
                     sheet_name='Sheet1')

print('Data loaded into a dataframe!')

Data loaded into a dataframe!


let's take a look

In [3]:
df_hd_RJ.head()

,Cod_ID,UDH_Atlas,NOME_UDH,CODMUN6,NOME_MUN,CODUF,NOME_UF,CODRM,NOME_RM,ANO,...,T_FLFUND_tudo,T_FLMED_tudo,T_FLBAS_tudo,T_FUND11A13_tudo,I_ESCOLARIDADE,I_FREQ_PROP,IDHM,IDHM_E,IDHM_L,IDHM_R
0,1330045600001,1330045600001,Vila Heliópolis,330045,Belford Roxo (RJ),33,RJ,63300,RM Rio Janeiro (RJ),2000,...,87.86,19.34,84.37,68.86,0.309,0.406,0.533,0.371,0.701,0.583
1,1330045600002,1330045600002,Santa Amélia / Barro Vermelho,330045,Belford Roxo (RJ),33,RJ,63300,RM Rio Janeiro (RJ),2000,...,91.11,25.09,86.62,65.99,0.342,0.446,0.556,0.408,0.708,0.595
2,1330045600003,1330045600003,Parque Fluminense / Vila Esperança,330045,Belford Roxo (RJ),33,RJ,63300,RM Rio Janeiro (RJ),2000,...,90.96,23.88,88.57,57.09,0.327,0.384,0.537,0.364,0.710,0.598
3,1330045600004,1330045600004,Parque dos Califas / Parque Jordão / Parque Ro...,330045,Belford Roxo (RJ),33,RJ,63300,RM Rio Janeiro (RJ),2000,...,88.41,23.84,83.54,69.84,0.296,0.342,0.517,0.326,0.709,0.598
4,1330045600005,1330045600005,Heliópolis / Jardim América,330045,Belford Roxo (RJ),33,RJ,63300,RM Rio Janeiro (RJ),2000,...,91.30,35.39,87.25,75.31,0.398,0.468,0.582,0.443,0.724,0.616


In [4]:
print(df_hd_RJ.columns.values)

['Cod_ID' 'UDH_Atlas' 'NOME_UDH' 'CODMUN6' 'NOME_MUN' 'CODUF' 'NOME_UF'
 'CODRM' 'NOME_RM' 'ANO' 'ESPVIDA' 'FECTOT' 'MORT1' 'MORT5' 'SOBRE40'
 'SOBRE60' 'RAZDEP' 'T_ENV' 'E_ANOSESTUDO' 'T_ANALF11A14' 'T_ANALF15A17'
 'T_ANALF15M' 'T_ANALF18A24' 'T_ANALF18M' 'T_ANALF25A29' 'T_ANALF25M'
 'T_ATRASO_2_BASICO' 'T_ATRASO_2_FUND' 'T_FBBAS' 'T_FBFUND' 'T_FBMED'
 'T_FBSUPER' 'T_FLBAS' 'T_FLFUND' 'T_FLMED' 'T_FLSUPER' 'T_FREQ0A5'
 'T_FREQ15A17' 'T_FREQ18A24' 'T_FREQ25A29' 'T_FREQ5A6' 'T_FREQ6A14'
 'T_FREQ6A17' 'T_FREQFUND1517' 'T_FREQFUND1824' 'T_FREQMED1824'
 'T_FUND11A13' 'T_FUND15A17' 'T_FUND18A24' 'T_FUND18M' 'T_FUND25M'
 'T_MED18A20' 'T_MED18A24' 'T_MED18M' 'T_MED25M' 'T_SUPER25M' 'CORTE1'
 'CORTE2' 'CORTE3' 'CORTE4' 'CORTE9' 'GINI' 'PIND' 'PINDCRI' 'PMPOB'
 'PMPOBCRI' 'PPOB' 'PPOBCRI' 'PREN10RICOS' 'PREN20' 'PREN20RICOS' 'PREN40'
 'PREN60' 'PREN80' 'PRENTRAB' 'R1040' 'R2040' 'RDPC' 'RDPC1' 'RDPC10'
 'RDPC2' 'RDPC3' 'RDPC4' 'RDPC5' 'RDPCT' 'RIND' 'RMPOB' 'RPOB' 'THEIL'
 'P_AGRO' 'P_COM' 'P_C

Let's select the columns that are important for our work: NOME_UDH (neighborhood name), NOME_MUN (city name), ANO (year), GINI (gini index), PIND (proportion of extremely poor), RDPC (average per capita income), IDHM (human development index HDI), PESOTOT (total population residing in the neighborhood).

You can find the headers description in file 'A - DICIONÁRIO dos indicadores do Atlas' that comes in the zip file. Unfortunately is portuguese only, but you can translate it if you wish to take a better look.
 
We will change the headers later

In [6]:
df_data_RJ = df_hd_RJ[['NOME_UDH', 'NOME_MUN', 'ANO', 'GINI', 'PIND', 'RDPC', 'IDHM', 'PESOTOT']]
df_data_RJ.head()

,NOME_UDH,NOME_MUN,ANO,GINI,PIND,RDPC,IDHM,PESOTOT
0,Vila Heliópolis,Belford Roxo (RJ),2000,0.40,6.77,301.71,0.533,8954.0
1,Santa Amélia / Barro Vermelho,Belford Roxo (RJ),2000,0.42,6.97,324.09,0.556,16502.0
2,Parque Fluminense / Vila Esperança,Belford Roxo (RJ),2000,0.44,7.14,331.46,0.537,13833.0
3,Parque dos Califas / Parque Jordão / Parque Ro...,Belford Roxo (RJ),2000,0.42,4.44,329.95,0.517,13166.0
4,Heliópolis / Jardim América,Belford Roxo (RJ),2000,0.41,4.13,369.82,0.582,12780.0


Now let's change the columns headers to more intuitive ones. And verify if it's ok.

In [8]:
df_data_RJ.columns = ['Region', 'City', 'Year', 'Gini Index', '% Extremely Poor', 'Per Capita Income', 'HDI', 'Total Population'] 
df_data_RJ.head()

,Region,City,Year,Gini Index,% Extremely Poor,Per Capita Income,HDI,Total Population
0,Vila Heliópolis,Belford Roxo (RJ),2000,0.40,6.77,301.71,0.533,8954.0
1,Santa Amélia / Barro Vermelho,Belford Roxo (RJ),2000,0.42,6.97,324.09,0.556,16502.0
2,Parque Fluminense / Vila Esperança,Belford Roxo (RJ),2000,0.44,7.14,331.46,0.537,13833.0
3,Parque dos Califas / Parque Jordão / Parque Ro...,Belford Roxo (RJ),2000,0.42,4.44,329.95,0.517,13166.0
4,Heliópolis / Jardim América,Belford Roxo (RJ),2000,0.41,4.13,369.82,0.582,12780.0


Now, let's filter the rows for the year of 2010 and Rio de Janeiro City.

We can drop the colmuns Year and City now, because we only have one value for each one of them. But keep in mind that our work is limited to the city of Rio de Janeiro, and the data we are using was collected in 2010.


In [9]:

df_data_RJ = df_data_RJ.loc[(df_data_RJ['City'] == 'Rio de Janeiro (RJ)') & (df_data_RJ['Year'] == 2010)]

Let's convert the column Total Population to integer

In [10]:
df_data_RJ['Total Population'].astype(np.int64)

3088    26098
3089    28293
3090    14985
3091    33196
3092     1127
        ...  
4219      638
4220    35684
4221    29078
4222    30027
4223    28813
Name: Total Population, Length: 1136, dtype: int64

Let's change the region Recreio and rename it to Recreio dos Bandeirantes. After some processing I noticed this name was incomplete in this table.

In [13]:
df_data_RJ.iat[int(df_data_RJ[df_data_RJ['Region']=='Recreio'].index[0]),0] = 'Recreio dos Bandeirantes'

Let's check if it worked

In [14]:
df_data_RJ.loc[(df_data_RJ['Region'] == 'Recreio dos Bandeirantes')]

,Region,Gini Index,% Extremely Poor,Per Capita Income,HDI,Total Population
1102,Recreio dos Bandeirantes,0.51,NaN,4427.29,0.943,27234.0


Let's define a function that calculates the weighted average of each column using as weight an specific column, and return a dict with the average values and the total for the weight column.

In [15]:
def df_weighted_average(dataframe, columns_list, total_column):
    '''
    This function takes pandas dataframe and calculates a weighted average for the columns you want, using as weight a column of your choice.
    dataframe (pandas dataframe): Dataframe you want to calculate the weighted averages and the total column.
    columns_list (list): List containing the names of the columns you wish to have the weighted average calculated.
    total_column (str): Name of the  column to be used as weight.
    '''
    # create dict with using the dataframe header as keys and set items to be 0
    columns_dict = dict.fromkeys(dataframe.columns , 0)
    
    # iterate over the rows of the data frame, multiplying the each column value by the weight
    for index, row in dataframe.iterrows():
        for key in columns_list:
            if not np.isnan(row[key]): # ignore NaN
                columns_dict.update({key : columns_dict[key] + (row[key] * row[total_column])})
        
        # calculates the total weight
        columns_dict.update({total_column : columns_dict[total_column] + row[total_column]})
    
    # replaces the items which are 0 for '' and calculates the weighted average
    for key in columns_dict:
        if columns_dict[key] != 0 and key != total_column:
            columns_dict.update({key : round(columns_dict[key] / columns_dict[total_column], 2)})
        elif columns_dict[key] == 0:
            columns_dict.update({key : ''})
             
    return columns_dict      
               
    

In [16]:
df_data_RJ.columns

Index(['Region', 'Gini Index', '% Extremely Poor', 'Per Capita Income', 'HDI',
       'Total Population'],
      dtype='object')

Now, we can calculate the weighted average.

In [17]:
df_data_RJ = df_data_RJ.append(df_weighted_average(df_data_RJ, ['Gini Index', '% Extremely Poor', 'Per Capita Income', 'HDI'], 'Total Population'), ignore_index=True)


In [18]:
print(df_weighted_average(df_data_RJ, ['Gini Index', '% Extremely Poor', 'Per Capita Income', 'HDI'], 'Total Population'))

{'Region': '', 'Gini Index': 0.44, '% Extremely Poor': 1.25, 'Per Capita Income': 1500.49, 'HDI': 0.79, 'Total Population': 12640892.0}


We can see that 1.25% of the population is considered extremely poor. 

After trying to get coordinates for the neighbourhoods, I notice 'Braz de Pina' was misspelled, the correct is Bras de Pina. So we use the code below to generate a corrected json.

In [19]:
with open("geoJson_RJ.json", "r") as old:
    RJ_neighbourhoods = json.load(old)
    for neighbourhood in RJ_neighbourhoods['features']:
        if neighbourhood['properties']['BAIRRO'] == 'Braz de Pina':
            neighbourhood['properties']['BAIRRO'] = 'Bras de Pina'

# dump json to another file
with open("new_geoJson_RJ.json", "w") as new:
    new.write(json.dumps(RJ_neighbourhoods))

Now, let's create a list of neighborhoods using the geoJson file

In [20]:
RJ_neighbourhoods = []
with open('new_geoJson_RJ.json') as data_file:    
    RJ = json.load(data_file)
    for neighbourhood in RJ['features']:
        RJ_neighbourhoods.append(neighbourhood['properties']['BAIRRO'])
        
RJ_neighbourhoods.sort()        
print(RJ_neighbourhoods)
print('')
print('There are ' + str(len(RJ_neighbourhoods)) + ' neighbourhoods in Rio de Janeiro')

['Abolição', 'Acari', 'Alto da Boa Vista', 'Anchieta', 'Andaraí', 'Anil', 'Bancários', 'Bangu', 'Barra da Tijuca', 'Barra de Guaratiba', 'Barros Filho', 'Benfica', 'Bento Ribeiro', 'Bonsucesso', 'Botafogo', 'Bras de Pina', 'Cachambi', 'Cacuia', 'Caju', 'Camorim', 'Campinho', 'Campo Grande', 'Campo dos Afonsos', 'Cascadura', 'Catete', 'Catumbi', 'Cavalcanti', 'Centro', 'Cidade Nova', 'Cidade Universitária', 'Cidade de Deus', 'Cocotá', 'Coelho Neto', 'Colégio', 'Complexo do Alemão', 'Copacabana', 'Cordovil', 'Cosme Velho', 'Cosmos', 'Costa Barros', 'Curicica', 'Del Castilho', 'Deodoro', 'Encantado', 'Engenheiro Leal', 'Engenho Novo', 'Engenho da Rainha', 'Engenho de Dentro', 'Estácio', 'Flamengo', 'Freguesia (Ilha)', 'Freguesia (Jacarepaguá)', 'Galeão', 'Gamboa', 'Gardênia Azul', 'Gericinó', 'Glória', 'Grajaú', 'Grumari', 'Guadalupe', 'Guaratiba', 'Gávea', 'Higienópolis', 'Honório Gurgel', 'Humaitá', 'Imperial de São Cristóvão', 'Inhaúma', 'Inhoaíba', 'Ipanema', 'Irajá', 'Itanhangá', 'Ja

We can see that the number of neighbourhoods is less much smaller than the number of regions in the human development table. This is due to the fact that the some regions are subdivisions of the actual neighbourhoods.

In this work, we will assume that the main region with the official neighbourhood name in the human development table is a good sample of the whole neighbourhood. And, for this reason, we will not use the total popultaion anymore.  

That beind said, we now can create a new dataframe containing the human development data for each neighbourhood. Later we will use geopy to get the coordinates of each neighbourhood and insert it to the dataframe.

Let's  iterate over human development dataframe and create dictionaries using regions names as keys and the other the other informations as numbers.

In [22]:
dict_human_development = {}

for index, row in df_data_RJ.iterrows():
    dict_human_development.update({row[0] : {'Gini Index' : row[1], '% Extremely Poor' : row[2], 'Per Capita Income' : row[3], 'Human Development Index' : row[4], 'Total Population' : row[5]}})


Let's create a dictionary with the coordinates of the each neighbourhood in the list we got from the geojson file.

In [ ]:
dict_latitude_longitude = {}

for item in RJ_neighbourhoods:
    address = str(item) + ', Rio de Janeiro, RJ'    
    print(address)
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    try:        
        dict_latitude_longitude.update({item : {'latitude' : location.latitude, 'longitude' : location.longitude}})
    except:
        print('!!!!!!!!!!!!!!!!!Coordinates of ' + item + ' not found!!!!!!!!!!!!!!!!!!!!!!')

print(dict_latitude_longitude)

I decided to save the dictionary into a json so we don't need to rely on geopy from now on.

In [ ]:
with open('dict_latitude_longitude.json', 'w') as json_file:
    json.dump(dict_latitude_longitude, json_file)

Now lets reload the json data into the dctionary

In [23]:
with open("dict_latitude_longitude.json", "r") as dic_coordinates:
    dict_coo = json.load(dic_coordinates)

dict_latitude_longitudes = dict_coo.copy()


Let's now try to match the neighbourhood names to the names in the human development table.

In [24]:

list_Neighbourhood =  []
list_Latitude = []
list_Longitude = []
list_gini_index = []
list_extremely_poor = []
list_per_capita_income = []
list_hdi = []


for key in dict_latitude_longitudes:
    
    total_population = 0
    gini_index = 0
    extremely_poor = 0 
    per_capita_income = 0
    hdi = 0
    
    list_Neighbourhood.append(key)
    list_Latitude.append(dict_latitude_longitudes[key]['latitude'])
    list_Longitude.append(dict_latitude_longitudes[key]['longitude'])
        
    for name in dict_human_development:
        if str(unidecode(key.split(' (')[0]).lower()) in str(unidecode(name).lower()):
            total_population += dict_human_development[name].get('Total Population')
            gini_index += dict_human_development[name].get('Gini Index') * dict_human_development[name].get('Total Population')
            if not np.isnan(dict_human_development[name].get('% Extremely Poor')):
                extremely_poor += dict_human_development[name].get('% Extremely Poor') * dict_human_development[name].get('Total Population')
            per_capita_income += dict_human_development[name].get('Per Capita Income') * dict_human_development[name].get('Total Population')
            hdi += dict_human_development[name].get('Human Development Index') * dict_human_development[name].get('Total Population')
    
    if total_population == 0:
        list_gini_index.append('')
        list_extremely_poor.append('')
        list_per_capita_income.append('')
        list_hdi.append('') 
    else:
        list_gini_index.append(round(gini_index / total_population, 2))
        list_extremely_poor.append(round(extremely_poor / total_population, 2))
        list_per_capita_income.append(round(per_capita_income / total_population, 2))
        list_hdi.append(round(hdi / total_population, 3)) 
            
consolidated_data = {'Neighbourhood': list_Neighbourhood, 'Latitude': list_Latitude, 'Longitude' : list_Longitude, 'Gini Index' : list_gini_index, '% Extremely Poor' : list_extremely_poor, \
                     'Per Capita Income' : list_per_capita_income, 'HDI' : list_hdi}
df_neighbourhoods_HD_RJ = pd.DataFrame(consolidated_data, columns = ['Neighbourhood', 'Latitude', 'Longitude', 'Gini Index', '% Extremely Poor', 'Per Capita Income', 'HDI'])

pd.set_option('display.max_rows', 170)
df_neighbourhoods_HD_RJ


,Neighbourhood,Latitude,Longitude,Gini Index,% Extremely Poor,Per Capita Income,HDI
0,Abolição,-22.886161,-43.299846,0.45,0.65,1201.72,0.833
1,Acari,-22.822153,-43.340674,0.41,3.24,535.21,0.685
2,Alto da Boa Vista,-22.962113,-43.253582,0.52,0.02,3555,0.925
3,Anchieta,-22.823190,-43.399107,0.45,1.14,859.53,0.773
4,Andaraí,-22.929084,-43.253486,0.44,0.66,1163.26,0.788
5,Anil,-22.955813,-43.338412,0.51,0.3,2063.47,0.862
6,Bancários,-22.791759,-43.180966,0.46,0.28,1537.15,0.857
7,Bangu,-22.875305,-43.464880,0.44,0.8,967.14,0.789
8,Barra da Tijuca,-22.999740,-43.365993,,,,
9,Barra de Guaratiba,-23.036004,-43.552140,0.42,1.7,661.33,0.744


Now let's select the neighbourhoods that weren't filled with data from the other table so we can try to fill them manually

In [25]:
df_neighbourhoods_HD_RJ_unfilled = df_neighbourhoods_HD_RJ.loc[(df_neighbourhoods_HD_RJ['HDI'] == '')]

df_neighbourhoods_HD_RJ_unfilled

,Neighbourhood,Latitude,Longitude,Gini Index,% Extremely Poor,Per Capita Income,HDI
8,Barra da Tijuca,-22.999740,-43.365993,,,,
13,Bonsucesso,-22.865932,-43.253318,,,,
39,Costa Barros,-22.824700,-43.369839,,,,
62,Higienópolis,-22.873999,-43.261368,,,,
65,Imperial de São Cristóvão,-22.893679,-43.220578,,,,
111,Pitangueiras,-22.817128,-43.179954,,,,
116,Quintino Bocaiúva,-22.895589,-43.325327,,,,
160,Zumbi,-22.821278,-43.175861,,,,


We will now create a dictionary to handle these neighbourhoods, with exception for Recreio dos Bandeirantes, wich we will correct in df_data_RJ, because the word Recreio appears in heighbourhoods that have no relation to Recreio dos Bandeirantes



Mapping:

    Barra da Tijuca --> Jardim Oceânico,  Américas / Marapendi, Sernambetiba 
    
    Bonsucesso is Bonsussesso
    
    Costa Barros --> Camboatá
    
    Higienópolis --> Higianópolis
    
    Imperial de São Cristóvão --> São Cristóvão
    
    Pitangueiras --> Ribeira
    
    Quintino Bocaiúva --> Quintino
    
    Recreio dos Bandeirantes --> Recreio
    
    Zumbi --> Ribeira



In [27]:
dict_neighbourhoods_exceptions = {'Barra da Tijuca':['Jardim Oceânico','Américas / Marapendi', 'Sernambetiba'], 'Bonsucesso':['Bonsussesso'], 'Costa Barros':['Camboatá'], \
                                  'Higienópolis':['Higianópolis'], 'Imperial de São Cristóvão':['São Cristóvão'], 'Pitangueiras':['Ribeira'], 'Quintino Bocaiúva':['Quintino'], \
                                  'Zumbi': ['Ribeira']}

Now we will rewrite and then rerun the script used to consolidate the neighbourhoods human development data with the neighbourhood data.

In [28]:
list_Neighbourhood =  []
list_Latitude = []
list_Longitude = []
list_gini_index = []
list_extremely_poor = []
list_per_capita_income = []
list_hdi = []


for key in dict_latitude_longitudes:
    
    total_population = 0
    gini_index = 0
    extremely_poor = 0 
    per_capita_income = 0
    hdi = 0
    
    list_Neighbourhood.append(key)
    list_Latitude.append(dict_latitude_longitudes[key]['latitude'])
    list_Longitude.append(dict_latitude_longitudes[key]['longitude'])
    
    if key in dict_neighbourhoods_exceptions:
        for item in dict_neighbourhoods_exceptions[key]:
            for name in dict_human_development:
                if str(unidecode(item).lower()) in str(unidecode(name).lower()):
                    total_population += dict_human_development[name].get('Total Population')
                    gini_index += dict_human_development[name].get('Gini Index') * dict_human_development[name].get('Total Population')
                    if not np.isnan(dict_human_development[name].get('% Extremely Poor')):
                        extremely_poor += dict_human_development[name].get('% Extremely Poor') * dict_human_development[name].get('Total Population')
                    per_capita_income += dict_human_development[name].get('Per Capita Income') * dict_human_development[name].get('Total Population')
                    hdi += dict_human_development[name].get('Human Development Index') * dict_human_development[name].get('Total Population')
            
    
    else:
        for name in dict_human_development:
            if str(unidecode(key.split(' (')[0]).lower()) in str(unidecode(name).lower()):
                total_population += dict_human_development[name].get('Total Population')
                gini_index += dict_human_development[name].get('Gini Index') * dict_human_development[name].get('Total Population')
                if not np.isnan(dict_human_development[name].get('% Extremely Poor')):
                    extremely_poor += dict_human_development[name].get('% Extremely Poor') * dict_human_development[name].get('Total Population')
                per_capita_income += dict_human_development[name].get('Per Capita Income') * dict_human_development[name].get('Total Population')
                hdi += dict_human_development[name].get('Human Development Index') * dict_human_development[name].get('Total Population')
                       
    
    if total_population == 0:
        list_gini_index.append('')
        list_extremely_poor.append('')
        list_per_capita_income.append('')
        list_hdi.append('') 
    else:
        list_gini_index.append(round(gini_index / total_population, 2))
        list_extremely_poor.append(round(extremely_poor / total_population, 2))
        list_per_capita_income.append(round(per_capita_income / total_population, 2))
        list_hdi.append(round(hdi / total_population, 3)) 
            
consolidated_data = {'Neighbourhood': list_Neighbourhood, 'Latitude': list_Latitude, 'Longitude' : list_Longitude, 'Gini Index' : list_gini_index, '% Extremely Poor' : list_extremely_poor, \
                     'Per Capita Income' : list_per_capita_income, 'HDI' : list_hdi}
df_neighbourhoods_HD_RJ = pd.DataFrame(consolidated_data, columns = ['Neighbourhood', 'Latitude', 'Longitude', 'Gini Index', '% Extremely Poor', 'Per Capita Income', 'HDI'])

pd.set_option('display.max_rows', 170)
df_neighbourhoods_HD_RJ



,Neighbourhood,Latitude,Longitude,Gini Index,% Extremely Poor,Per Capita Income,HDI
0,Abolição,-22.886161,-43.299846,0.45,0.65,1201.72,0.833
1,Acari,-22.822153,-43.340674,0.41,3.24,535.21,0.685
2,Alto da Boa Vista,-22.962113,-43.253582,0.52,0.02,3555.00,0.925
3,Anchieta,-22.823190,-43.399107,0.45,1.14,859.53,0.773
4,Andaraí,-22.929084,-43.253486,0.44,0.66,1163.26,0.788
5,Anil,-22.955813,-43.338412,0.51,0.30,2063.47,0.862
6,Bancários,-22.791759,-43.180966,0.46,0.28,1537.15,0.857
7,Bangu,-22.875305,-43.464880,0.44,0.80,967.14,0.789
8,Barra da Tijuca,-22.999740,-43.365993,0.48,0.32,5411.65,0.941
9,Barra de Guaratiba,-23.036004,-43.552140,0.42,1.70,661.33,0.744


Let's divide the neighbourhoods in 4 clusters. But, first we have to normalize the data using StandardScaler().

In [30]:
X = df_neighbourhoods_HD_RJ.values[:,3:]
Clus_dataSet = StandardScaler().fit_transform(X)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
clusterNum = 4
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 30)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[1 1 2 1 1 3 3 1 0 1 1 1 1 1 2 1 3 3 1 3 1 1 3 1 3 1 3 1 3 1 1 3 1 1 1 2 1
 2 1 1 1 1 1 1 1 3 1 3 1 0 3 3 3 1 1 1 3 3 1 1 1 0 1 1 2 3 1 1 0 1 2 0 1 3
 1 0 1 3 3 3 0 1 2 0 0 3 1 1 1 1 2 1 1 1 3 3 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1
 3 3 3 3 3 1 1 1 2 2 3 1 3 1 1 1 1 1 1 1 1 1 1 1 1 0 3 1 1 1 2 3 1 1 2 1 3
 1 1 1 1 1 3 1 1 1 3 3 1 3 1]


In [32]:
df_neighbourhoods_HD_RJ['Cluster Labels'] = labels
df_neighbourhoods_HD_RJ.head()

,Neighbourhood,Latitude,Longitude,Gini Index,% Extremely Poor,Per Capita Income,HDI,Cluster Labels
0,Abolição,-22.886161,-43.299846,0.45,0.65,1201.72,0.833,1
1,Acari,-22.822153,-43.340674,0.41,3.24,535.21,0.685,1
2,Alto da Boa Vista,-22.962113,-43.253582,0.52,0.02,3555.00,0.925,2
3,Anchieta,-22.823190,-43.399107,0.45,1.14,859.53,0.773,1
4,Andaraí,-22.929084,-43.253486,0.44,0.66,1163.26,0.788,1


We can easily check the centroid values by averaging the features in each cluster.

In [33]:
df_neighbourhoods_HD_RJ.groupby('Cluster Labels').mean()

,Latitude,Longitude,Gini Index,% Extremely Poor,Per Capita Income,HDI
Cluster Labels,,,,,,
0,-22.972833,-43.243764,0.508000,0.099000,6373.319000,0.944200
1,-22.875902,-43.337362,0.432600,1.214900,872.046200,0.763210
2,-22.951454,-43.238485,0.484167,0.146667,4136.088333,0.935583
3,-22.882545,-43.262431,0.462500,0.302750,2035.301250,0.870650


Now let's associate a color for each label. We will tag them according to their per capita income centroid values. 

In [67]:
df_centroids = df_neighbourhoods_HD_RJ.groupby('Cluster Labels').mean().sort_values(by=['Per Capita Income'])
df_centroids['labels'] = df_centroids.index
df_centroids = df_centroids.reset_index(drop=True)

dict_label_colors = {}

for index, row in df_centroids.iterrows():
    dict_label_colors.update({df_centroids['labels'][index] : index})

cluster_color = []
for index, row in df_neighbourhoods_HD_RJ.iterrows():
    if dict_label_colors[df_neighbourhoods_HD_RJ['Cluster Labels'][index]] == 0:
        cluster_color.append('red')
    elif dict_label_colors[df_neighbourhoods_HD_RJ['Cluster Labels'][index]] == 1:
        cluster_color.append('orange')
    elif dict_label_colors[df_neighbourhoods_HD_RJ['Cluster Labels'][index]] == 2:
        cluster_color.append('green')
    elif dict_label_colors[df_neighbourhoods_HD_RJ['Cluster Labels'][index]] == 3:
        cluster_color.append('blue')
    
    
df_neighbourhoods_HD_RJ['Cluster Colors'] = cluster_color
df_neighbourhoods_HD_RJ.head()



,Neighbourhood,Latitude,Longitude,Gini Index,% Extremely Poor,Per Capita Income,HDI,Cluster Labels,Cluster Colors
0,Abolição,-22.886161,-43.299846,0.45,0.65,1201.72,0.833,1,red
1,Acari,-22.822153,-43.340674,0.41,3.24,535.21,0.685,1,red
2,Alto da Boa Vista,-22.962113,-43.253582,0.52,0.02,3555.00,0.925,2,green
3,Anchieta,-22.823190,-43.399107,0.45,1.14,859.53,0.773,1,red
4,Andaraí,-22.929084,-43.253486,0.44,0.66,1163.26,0.788,1,red


Let's create cloropleth map using Gini Index, using different colors markers to identify the clusters

In [36]:
latitude = -22.8945945
longitude = -43.429157
 
RJ_geo = r'new_geoJson_RJ.json'

# let Folium determine the scale.
RJ_gini_map = folium.Map(location=[latitude, longitude], width=1200, height=650, zoom_start=11)
RJ_gini_map.choropleth(
    geo_data=RJ_geo,
    data=df_neighbourhoods_HD_RJ,
    columns=['Neighbourhood', 'Gini Index'],
    key_on='feature.properties.BAIRRO',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.7,
    legend_name='Gini Index of Rio de Janeiro Neighbourhoods',
    reset=True
)


for index, row in df_neighbourhoods_HD_RJ.iterrows():
    label = str(df_neighbourhoods_HD_RJ['Neighbourhood'][index]) + '  |  Gini Index: ' + str(df_neighbourhoods_HD_RJ['Gini Index'][index])
    folium.Marker(
        location=[df_neighbourhoods_HD_RJ['Latitude'][index], df_neighbourhoods_HD_RJ['Longitude'][index]],
        popup=label,
        icon=folium.Icon(color=df_neighbourhoods_HD_RJ['Cluster Colors'][index])
    ).add_to(RJ_gini_map)


RJ_gini_map.save('gini_map.html') #saves the map to a html file


In [7]:
IFrame(src='./gini_map.html', width=1220, height=670) # render the map we've created

We can observe that in general the neighbourhoods of the blue and green clusters are more unequal according to Gini Index.

In [38]:
latitude = -22.8945945
longitude = -43.429157

RJ_geo = r'new_geoJson_RJ.json'

# let Folium determine the scale.
RJ_extremely_poor_map = folium.Map(location=[latitude, longitude], width=1200, height=650, zoom_start=11)
RJ_extremely_poor_map.choropleth(
    geo_data=RJ_geo,
    data=df_neighbourhoods_HD_RJ,
    columns=['Neighbourhood', '% Extremely Poor'],
    key_on='feature.properties.BAIRRO',
    fill_color='BuPu', 
    fill_opacity=0.7, 
    line_opacity=0.7,
    legend_name='% Extremely Poor of Rio de Janeiro by Neighbourhood',
    reset=True
)


for index, row in df_neighbourhoods_HD_RJ.iterrows():
    label = str(df_neighbourhoods_HD_RJ['Neighbourhood'][index]) + '  |  % Extremely Poor: ' + str(df_neighbourhoods_HD_RJ['% Extremely Poor'][index])
    folium.Marker(
        location=[df_neighbourhoods_HD_RJ['Latitude'][index], df_neighbourhoods_HD_RJ['Longitude'][index]],
        popup=label,
        icon=folium.Icon(color=df_neighbourhoods_HD_RJ['Cluster Colors'][index])
    ).add_to(RJ_extremely_poor_map)


RJ_extremely_poor_map.save('extremely_poor_map.html') #saves the map to a html file

In [6]:
IFrame(src='./extremely_poor_map.html', width=1220, height=670) # render the map we've created

We can see in this map that probably the purple areas are good choices of location to open community refectories.

In [40]:
latitude = -22.8945945
longitude = -43.429157

RJ_geo = r'new_geoJson_RJ.json'

# let Folium determine the scale.
RJ_per_capita_income_map = folium.Map(location=[latitude, longitude], width=1200, height=650, zoom_start=11)
RJ_per_capita_income_map.choropleth(
    geo_data=RJ_geo,
    data=df_neighbourhoods_HD_RJ,
    columns=['Neighbourhood', 'Per Capita Income'],
    key_on='feature.properties.BAIRRO',
    fill_color='GnBu', 
    fill_opacity=0.9, 
    line_opacity=0.7,
    legend_name='Per Capita Income Map of Rio de Janeiro by Neighbourhood',
    reset=True
)


for index, row in df_neighbourhoods_HD_RJ.iterrows():
    label = str(df_neighbourhoods_HD_RJ['Neighbourhood'][index]) + '  |  Per Capita Income: ' + str(df_neighbourhoods_HD_RJ['Per Capita Income'][index])
    folium.Marker(
        location=[df_neighbourhoods_HD_RJ['Latitude'][index], df_neighbourhoods_HD_RJ['Longitude'][index]],
        popup=label,
        icon=folium.Icon(color=df_neighbourhoods_HD_RJ['Cluster Colors'][index])
    ).add_to(RJ_per_capita_income_map)


RJ_per_capita_income_map.save('per_capita_income_map.html') #saves the map to a html file

In [5]:
IFrame(src='./per_capita_income_map.html', width=1220, height=670) # render the map we've created

In [42]:
latitude = -22.8945945
longitude = -43.429157

RJ_geo = r'new_geoJson_RJ.json'

# let Folium determine the scale.
RJ_hdi_map = folium.Map(location=[latitude, longitude], width=1200, height=650, zoom_start=11)
RJ_hdi_map.choropleth(
    geo_data=RJ_geo,
    data=df_neighbourhoods_HD_RJ,
    columns=['Neighbourhood', 'HDI'],
    key_on='feature.properties.BAIRRO',
    fill_color='PuBuGn', 
    fill_opacity=0.8, 
    line_opacity=0.7,
    legend_name='Human Development Index Map of Rio de Janeiro by Neighbourhood',
    reset=True
)


for index, row in df_neighbourhoods_HD_RJ.iterrows():
    label = str(df_neighbourhoods_HD_RJ['Neighbourhood'][index]) + '  |  Human Development Index: ' + str(df_neighbourhoods_HD_RJ['HDI'][index])
    folium.Marker(
        location=[df_neighbourhoods_HD_RJ['Latitude'][index], df_neighbourhoods_HD_RJ['Longitude'][index]],
        popup=label,
        icon=folium.Icon(color=df_neighbourhoods_HD_RJ['Cluster Colors'][index])
    ).add_to(RJ_hdi_map)


RJ_hdi_map.save('hdi_map.html') #saves the map to a html file

In [4]:
IFrame(src='./hdi_map.html', width=1220, height=670) # render the map we've created

We can see the human development index varies widely between neighbourhoods. This is primarily due to the fact the income distribution is very unequal across the city. Guaratiba is one of the most populated areas and have really bad social indicators.

Even though this data was collected in 2010, we have reasons to believe that the outlook is probably very similar nowadays. As mentioned before, Rio de Janeiro is going through a fiscal crisis, and this took a toll in the city's the social policies. And not only that, Brazil is facing the same problem and we saw the poverty levels increase in the last years.

https://translate.google.com/translate?sl=pt&tl=en&u=https%3A%2F%2Fg1.globo.com%2Feconomia%2Fnoticia%2F2018%2F12%2F05%2Fem-1-ano-aumenta-em-quase-2-milhoes-numero-de-brasileiros-em-situacao-de-pobreza-diz-ibge.ghtml

https://translate.google.com/translate?sl=pt&tl=en&u=https%3A%2F%2Fg1.globo.com%2Feconomia%2Fnoticia%2F2019%2F04%2F05%2Fbanco-mundial-alerta-para-aumento-da-pobreza-no-brasil.ghtml

https://translate.google.com/translate?sl=pt&tl=en&u=https%3A%2F%2Fg1.globo.com%2Feconomia%2Fnoticia%2F2019%2F06%2F04%2Fbanco-mundial-piora-previsao-para-o-pib-do-brasil-e-para-o-crescimento-mundial-em-2019.ghtml

Now we can start to collect supermarket data from Foursquare.

In [44]:
CLIENT_ID = 'ZUWEK5SZAYK0XEQZW24GNQX2LL3F4XR1GG1H03T3EQFOGZOC' # your Foursquare ID
CLIENT_SECRET = 'G1RHRKX01IEY5NQWT2TB2TJ3CBK35J4OTEJB14NYDXEHDG04' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZUWEK5SZAYK0XEQZW24GNQX2LL3F4XR1GG1H03T3EQFOGZOC
CLIENT_SECRET:G1RHRKX01IEY5NQWT2TB2TJ3CBK35J4OTEJB14NYDXEHDG04


In [49]:
df_supermarkets = pd.DataFrame(columns=['name', 'lat', 'lng'])

for index, row in df_neighbourhoods_HD_RJ.iterrows():

    lat = df_neighbourhoods_HD_RJ['Latitude'][index]
    lng = df_neighbourhoods_HD_RJ['Longitude'][index]
    CATEGORYID = '52f2ab2ebcbc57f1066b8b46'
    radius = 5000
    
    #print(str(lat) + ' , ' + str(lng))
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&categoryId={}&ll={},{}&radius={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        CATEGORYID,
        lat,
        lng,
        radius)


    results = requests.get(url).json()

    try:
    
        venues = results['response']['groups'][0]['items']

        temp_nearby_supermarkets = json_normalize(venues) # flatten JSON

        # filter columns
        filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng']
        temp_nearby_supermarkets = temp_nearby_supermarkets.loc[:, filtered_columns]

        # clean columns
        temp_nearby_supermarkets.columns = [col.split(".")[-1] for col in temp_nearby_supermarkets.columns]
        
        temp_copy_nearby_supermarkets = temp_nearby_supermarkets.copy()
        df_supermarkets = df_supermarkets.append(temp_copy_nearby_supermarkets, ignore_index = True)
    
        del temp_nearby_supermarkets
    
    except:
        print(df_neighbourhoods_HD_RJ['Neighbourhood'][index] + ' not found by Foursquare')

df_supermarkets = df_supermarkets.drop_duplicates().reset_index(drop=True)
df_supermarkets.head()

Grumari not found by Foursquare
Paquetá not found by Foursquare
Pedra de Guaratiba not found by Foursquare
Piedade not found by Foursquare


,name,lat,lng
0,Assaí Atacadista,-22.904614,-43.286328
1,Rede Economia de Supermercados,-22.876513,-43.268240
2,Assaí Atacadista,-22.883098,-43.290767
3,Supermercados Guanabara,-22.882244,-43.339348
4,Supermarket,-22.879090,-43.294873


In [50]:
print('{} supermarkets were returned by Foursquare.'.format(df_supermarkets.shape[0]))

329 supermarkets were returned by Foursquare.


In [64]:
latitude = -22.8945945
longitude = -43.429157

# let Folium determine the scale.
RJ_supermarkets_map = folium.Map(location=[latitude, longitude], width=1200, height=650, zoom_start=11)

for index, row in df_supermarkets.iterrows():
    label = str(df_supermarkets['name'][index]) 
    folium.Marker(
        location=[df_supermarkets['lat'][index], df_supermarkets['lng'][index]],
        popup=folium.Popup(label, parse_html=True, max_width=100),
        icon=folium.Icon(color='blue', icon='glyphicon-shopping-cart')
    ).add_to(RJ_supermarkets_map)
    
RJ_supermarkets_map.save('supermarkets_map.html') #saves the map to a html file


In [3]:
IFrame(src='./supermarkets_map.html', width=1220, height=670) # render the map we've created

We can see that "Guaratiba" and it's surroundings have few supermarket options, and earlier we saw that this same area is one with the highest extremely poor people. In this work, we believe this area should be the first contemplated with a community refectory, due to it's poor social indicators, the high populated area and low amount of supermarkets. 

The supermarkets of the city, could sell products near to expiration dates, or any other foods and products that otherwise would be wasted, but are still proper for consumption, to the community refectory. And the city hall could buy theses products at discount and sell meals for affordable prices for the population in need. A register could be created to keep control of who could have the benefit of using the community refectories.

The supermarkets would also benefit from this, because they can sell products that otherwise would be wasted and they could also advertise that they joined the program. 

After the system is established, the supermarket could also reserve a little space to receive products of clients that they wouldn't consume but are proper for consumption.

# 4. Conclusion <a id="6"></a>

We conclude that the first area to open the community restaurant should be Guaratiba, or a neighbouhood in it's surroundings that also have poor social indicators.

The supermarkets would be stimulated to work as partners of the city hall in this  project, as they can diminish their losses with waste and can advertise they participate in program that would have such a positive impact. 

In this work we could see that poverty and income inequality are a big issue to be faced in Rio de Janeiro.

# 5. Further Directions <a id="8"></a>

A similar system in which the city partner up with the restaurants as well could be studied.